In [1]:
var moment = require('moment')
var { constants } = require('../utils')
var { advanceTimeAndBlock } = require('../utils/evm')
var { evm } = require('../utils');
var { toUnit } = require('../utils/bn')
var { getLatestBlockTimestamp, getBlockTimestamp } = require('../utils/evm')
var { next, clear, bnToNumber } = require('../utils/jupyter')
var { Notebook } = require('../utils/notebook')
var { Keep3r } = require('../utils/common')
var { ethers } = require('hardhat')

oneHundred = toUnit(100)

var $ = new Notebook();
var keep3r = new Keep3r();
//var snapshots = new evm.SnapshotManager()

clear();

In [2]:
next(async () => {
    await $.setup(constants.FORK_BLOCK_NUMBER);
    await keep3r.setup();
    
    job = await(await ethers.getContractFactory('JobForTest')).deploy(keep3r.v2.address)
    provider = await $.newSigner();
    weth = await $.fetch('ERC20ForTest', constants.WETH_ADDRESS)
    uniV3Pool = await $.fetch('IUniswapV3Pool', constants.UNISWAP_V3_ORACLE_POOL )
});

In [3]:
next(async()=>{   
    tickTime = $.time(5,'days')
    uniResponse = await uniV3Pool.observe([0,tickTime])
    poolLiquidity = await uniV3Pool.liquidity()
    
    quote = 1.0001**(((uniResponse[0][1]).sub(uniResponse[0][0])).div(tickTime))
    
    console.log('previous liquidity', bnToNumber(poolLiquidity))
    console.log('previous tickQuote', 1/quote)
})

In [ ]:
next(async()=>{
    // mints 100 KP3R and 100 WETH to provider and approves spendings
    await keep3r.proxy.connect(keep3r.governance)['mint(address,uint256)'](provider.address, oneHundred)
    await keep3r.v1.connect(provider).approve(keep3r.pool.address, oneHundred)
    await weth.connect(provider).deposit(oneHundred, {value:oneHundred})
    await weth.connect(provider).approve(keep3r.pool.address, oneHundred)
    
    kp3rInitialBalance = await keep3r.v1.balanceOf(provider.address)
    wethInitialBalance = await weth.balanceOf(provider.address)

    // MINT
    liquidity = await keep3r.pool.connect(provider).callStatic.mint(oneHundred,oneHundred,0,0,provider.address)
    await keep3r.pool.connect(provider).mint(oneHundred,oneHundred,0,0,provider.address)
    
    kp3rBalance = await keep3r.v1.balanceOf(provider.address)
    wethBalance = await weth.balanceOf(provider.address)
    klpBalance = await keep3r.pool.balanceOf(provider.address)
    console.log('KP3R spent', bnToNumber(kp3rInitialBalance.sub(kp3rBalance)))
    console.log('WETH spent', bnToNumber(wethInitialBalance.sub(wethBalance)))
    console.log('kLP minted', bnToNumber(klpBalance))
})

In [5]:
// next(async()=>{
//     /* Notice:
//         at current block, pools cardinality is 1
//         provider have just minted a position in the pool
//         latest twap accumulator is 1s ago
//         UniswapV3Pool will return 'OLD' and provide no credits
//     */ 
//     await advanceTimeAndBlock(3 * 432000)
// })

In [6]:
next(async()=>{    
    await keep3r.v2.connect(provider).addJob(job.address)
    await keep3r.pool.connect(provider).approve(keep3r.v2.address, klpBalance)
    await keep3r.v2.connect(provider).addLiquidityToJob(job.address, keep3r.pool.address, klpBalance)
})

In [7]:
next(async()=>{
    console.log('totalJobCredits', (await keep3r.v2.totalJobCredits(job.address)).toString())
    console.log('jobPeriodCredits', (await keep3r.v2.jobPeriodCredits(job.address)).toString())
    console.log('jobLiquidityCredits', (await keep3r.v2.jobLiquidityCredits(job.address)).toString())
})

In [16]:
next(async()=>{
    tx = await job.connect(keep3r.keeper).workHard(10)
    wait = await tx.wait()
    console.log(await keep3r.v2.bonds(keep3r.keeper.address, keep3r.v1.address))
})

BigNumber { _hex: '0x00', _isBigNumber: true }


In [19]:
advanceTimeAndBlock(432000)

In [20]:
job.connect(keep3r.keeper).workHard(10)

{
  nonce: 10,
  gasPrice: BigNumber { _hex: '0x01dcd65000', _isBigNumber: true },
  gasLimit: BigNumber { _hex: '0xaf0a68', _isBigNumber: true },
  to: '0x5eb3Bc0a489C5A8288765d2336659EbCA68FCd00',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  data: '0x3bb39c33000000000000000000000000000000000000000000000000000000000000000a',
  chainId: 31337,
  v: 62710,
  r: '0x167d87ca7fc48879c7f62aec4ca4952b86e221e07df020011b24b1bf2db8834a',
  s: '0x4bc8e869ef297b5254c3fe0dc23a3fe863e8d1fa712b4ff736d54613a30e0e54',
  from: '0xD38C79984f938455A564988d86665c787eC47172',
  hash: '0x2a66e273537b1a00da033210d2bb6c32338bab466b0c77d8d4a422bc02889eaf',
  type: null,
  wait: [Function (anonymous)]
}

In [18]:
keep3r.v2.bonds(keep3r.keeper.address, keep3r.v1.address)

BigNumber { _hex: '0x00', _isBigNumber: true }